# Task 03: Adaptive Random Forest (ARF) – Giải pháp đề xuất

Triển khai **ARF** (thư viện River) với **7 biến thể** drift detector để giảm Catastrophic Forgetting và thích nghi concept drift trên NSL-KDD, không lưu toàn bộ dữ liệu lịch sử. Đánh giá **theo phase** và báo cáo **AA, FM, BWT** (context §5–6).

**Tham chiếu:** `exp_data/context/context_task_03.md`, `context_task_03_vie.md`

**Lưu ý:** Nếu chưa cài River, chạy lệnh sau trong terminal hoặc trong một cell: `pip install river`

## 1. Setup & Import

Dùng **River** cho ARF (online learning, drift detection tích hợp). Cấu hình đường dẫn và hằng số theo STRICT config từ context.

In [1]:
# Cell 1: Setup & Import
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

# --- STRICT CONFIG (context_task_03) ---
data_dir = Path(r'H:\tdc_window\Workspace\Testspace\ids\exp_data')
train_file = data_dir / 'KDDTrain+.txt'
test_file = data_dir / 'KDDTest+.txt'

categorical_cols = ['protocol_type', 'service', 'flag']
TARGET_CLASSES = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']
RANDOM_STATE = 42

standard_column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
    'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'attack_type', 'difficulty'
]
attack_categories = {
    'Normal': 'normal',
    'DoS': ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop', 'mailbomb', 'apache2', 'processtable', 'udpstorm'],
    'Probe': ['ipsweep', 'nmap', 'portsweep', 'satan', 'mscan', 'saint'],
    'R2L': ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster', 'sendmail', 'named', 'snmpgetattack', 'snmpguess', 'xlock', 'xsnoop', 'worm'],
    'U2R': ['buffer_overflow', 'loadmodule', 'perl', 'rootkit', 'httptunnel', 'ps', 'sqlattack', 'xterm']
}
print("Config loaded. TARGET_CLASSES:", TARGET_CLASSES)

Config loaded. TARGET_CLASSES: ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']


## 2. Load Data

Hàm load và map `attack_type` → `label` (5 lớp). Kiểm chứng: in `value_counts` và shape.

In [2]:
# Cell 2: Data Loading
def load_and_process_data(file_path, cols, mapping_dict, drop_difficulty=False):
    df = pd.read_csv(file_path, names=cols)
    df['attack_type'] = df['attack_type'].astype(str).str.strip().str.rstrip('.')
    reversed_mapping = {}
    for group, value in mapping_dict.items():
        if isinstance(value, list):
            for sub in value:
                reversed_mapping[sub] = group
        else:
            reversed_mapping[value] = group
    df['label'] = df['attack_type'].map(reversed_mapping).fillna('Unknown')
    if drop_difficulty:
        df = df.drop(columns=['difficulty'], errors='ignore')
    return df

train_df = load_and_process_data(train_file, standard_column_names, attack_categories)
test_df = load_and_process_data(test_file, standard_column_names, attack_categories)
print("train_df.shape:", train_df.shape)
print("test_df.shape:", test_df.shape)
print("\nTrain label counts:")
print(train_df['label'].value_counts())

train_df.shape: (125973, 44)
test_df.shape: (22544, 44)

Train label counts:
label
Normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: count, dtype: int64


## 3. Feature Engineering (Encoding)

Cùng pipeline Task 2: loại bỏ `label`, `attack_type`, `difficulty`; OneHotEncoder cho categorical; remainder passthrough → 122 chiều.

In [3]:
# Cell 3: Encoding
exclude_cols = ['label', 'attack_type', 'difficulty']
X_train = train_df.drop(columns=exclude_cols)
X_test = test_df.drop(columns=exclude_cols)
y_train = train_df['label']
y_test = test_df['label']

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
preprocessor = ColumnTransformer(
    transformers=[('cat', encoder, categorical_cols)],
    remainder='passthrough'
)
preprocessor.fit(X_train)
X_train_encoded = preprocessor.transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

print("X_train_encoded.shape:", X_train_encoded.shape)
print("X_test_encoded.shape:", X_test_encoded.shape)
print("y_train (first 3):", y_train.head(3).tolist())

X_train_encoded.shape: (125973, 122)
X_test_encoded.shape: (22544, 122)
y_train (first 3): ['Normal', 'Normal', 'DoS']


## 4. Phase Splitting

Phase 0 = Normal, DoS, Probe; Phase 1 = R2L only; Phase 2 = U2R only. Feed stream theo thứ tự 0 → 1 → 2.

In [4]:
# Cell 4: Phase split
mask_phase0 = y_train.isin(['Normal', 'DoS', 'Probe'])
mask_phase1 = (y_train == 'R2L')
mask_phase2 = (y_train == 'U2R')

X_phase0 = X_train_encoded[mask_phase0.values]
y_phase0 = y_train[mask_phase0].values
X_phase1 = X_train_encoded[mask_phase1.values]
y_phase1 = y_train[mask_phase1].values
X_phase2 = X_train_encoded[mask_phase2.values]
y_phase2 = y_train[mask_phase2].values

print("Phase 0:", X_phase0.shape[0], "samples")
print("Phase 1 (R2L):", X_phase1.shape[0], "samples")
print("Phase 2 (U2R):", X_phase2.shape[0], "samples")
print("Test set:", X_test_encoded.shape[0], "samples")

Phase 0: 124926 samples
Phase 1 (R2L): 995 samples
Phase 2 (U2R): 52 samples
Test set: 22544 samples


## 5. Helper: Convert sample to River dict

River dùng `learn_one(x, y)` với `x` là dict. Chuyển mỗi hàng (122 số) thành dict `{"f0": v0, "f1": v1, ...}`.

In [5]:
# Cell 5: River input format
def row_to_dict(row):
    return {f"f{i}": float(x) for i, x in enumerate(row)}

# Quick check
sample = row_to_dict(X_phase0[0])
print("Keys (first 5):", list(sample.keys())[:5])
print("Len:", len(sample))

Keys (first 5): ['f0', 'f1', 'f2', 'f3', 'f4']
Len: 122


## 6. Define ARF variants (River)

**Tham số chung:** `n_models=100`, `grace_period=50`. Bảy biến thể theo drift detector:
- 1) Base: không drift detector.
- 2) ADWIN: adaptive windowing, thay đổi trung bình (gradual drift).
- 3) KSWIN: Kolmogorov–Smirnov, thay đổi phân phối.
- 4) Page-Hinkley: thay đổi đột ngột trung bình (abrupt).
- 5) DDM: dựa trên tỷ lệ lỗi.
- 6) HDDM_A: Hoeffding A-test (moving average).
- 7) HDDM_W: Hoeffding W-test (weighted moving average).

In [6]:
# Cell 6: Build 7 ARF variants
from river import forest
from river import drift

N_MODELS = 100
GRACE_PERIOD = 50
SEED = RANDOM_STATE

def make_arf_base():
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=None, warning_detector=None)

def make_arf_adwin():
    d = drift.ADWIN(delta=0.001)
    w = drift.ADWIN(delta=0.01)
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=w)

def make_arf_kswin():
    d = drift.KSWIN(alpha=0.005)
    w = drift.KSWIN(alpha=0.01)
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=w)

def make_arf_ph():
    d = drift.PageHinkley()
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=None)

def make_arf_ddm():
    d = drift.DDM()
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=None)

def make_arf_hddm_a():
    d = drift.HDDM_A()
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=None)

def make_arf_hddm_w():
    d = drift.HDDM_W()
    return forest.ARFClassifier(n_models=N_MODELS, grace_period=GRACE_PERIOD, seed=SEED, drift_detector=d, warning_detector=None)

VARIANTS = [
    ('ARF_Base', make_arf_base),
    ('ARF_ADWIN', make_arf_adwin),
    ('ARF_KSWIN', make_arf_kswin),
    ('ARF_PH', make_arf_ph),
    ('ARF_DDM', make_arf_ddm),
    ('ARF_HDDM_A', make_arf_hddm_a),
    ('ARF_HDDM_W', make_arf_hddm_w),
]
print("Variants:", [v[0] for v in VARIANTS])

Variants: ['ARF_Base', 'ARF_ADWIN', 'ARF_KSWIN', 'ARF_PH', 'ARF_DDM', 'ARF_HDDM_A', 'ARF_HDDM_W']


## 7. Train stream & phase-wise evaluation (cho AA, FM, BWT)

Hàm `train_arf_stream`: feed Phase 0 → 1 → 2 (không đánh giá giữa chừng). Hàm `train_arf_stream_with_eval`: feed từng phase rồi đánh giá ngay trên test set, trả về `(report_after_p0, report_after_p1, report_after_p2)` để tính AA, FM, BWT (context §5).

In [7]:
# Cell 7: Train stream (Phase 0 -> 1 -> 2) + phase-wise eval for AA/FM/BWT
def train_arf_stream(model, X_phase0, y_phase0, X_phase1, y_phase1, X_phase2, y_phase2):
    for i in range(len(X_phase0)):
        model.learn_one(row_to_dict(X_phase0[i]), y_phase0[i])
    for i in range(len(X_phase1)):
        model.learn_one(row_to_dict(X_phase1[i]), y_phase1[i])
    for i in range(len(X_phase2)):
        model.learn_one(row_to_dict(X_phase2[i]), y_phase2[i])
    return model

def train_arf_stream_with_eval(model, X_phase0, y_phase0, X_phase1, y_phase1, X_phase2, y_phase2,
                               X_test_encoded, y_test):
    """Train phase-by-phase; evaluate after each phase. Returns (report_p0, report_p1, report_p2)."""
    for i in range(len(X_phase0)):
        model.learn_one(row_to_dict(X_phase0[i]), y_phase0[i])
    r0 = classification_report(y_test, predict_arf_batch(model, X_test_encoded),
                               labels=TARGET_CLASSES, output_dict=True, zero_division=0)
    for i in range(len(X_phase1)):
        model.learn_one(row_to_dict(X_phase1[i]), y_phase1[i])
    r1 = classification_report(y_test, predict_arf_batch(model, X_test_encoded),
                               labels=TARGET_CLASSES, output_dict=True, zero_division=0)
    for i in range(len(X_phase2)):
        model.learn_one(row_to_dict(X_phase2[i]), y_phase2[i])
    r2 = classification_report(y_test, predict_arf_batch(model, X_test_encoded),
                               labels=TARGET_CLASSES, output_dict=True, zero_division=0)
    return r0, r1, r2

print("train_arf_stream and train_arf_stream_with_eval defined.")

train_arf_stream and train_arf_stream_with_eval defined.


## 8. Predict on test set (batch)

Duyệt từng mẫu test, gọi `predict_one(x)` và thu thập dự đoán để dùng với `classification_report`.

In [8]:
# Cell 8: Predict batch
def predict_arf_batch(model, X_test_encoded):
    preds = []
    for i in range(len(X_test_encoded)):
        x = row_to_dict(X_test_encoded[i])
        p = model.predict_one(x)
        preds.append(p if p is not None else TARGET_CLASSES[0])
    return preds

print("Predict batch helper defined.")

Predict batch helper defined.


## 9. Train & evaluate one variant (ARF_Base) – phase-wise

Chạy thử ARF_Base với đánh giá sau từng phase (after_p0, after_p1, after_p2). In classification_report cuối để kiểm chứng.

In [9]:
# Cell 9: Train ARF_Base with phase-wise eval
model_base = make_arf_base()
rep0, rep1, rep2 = train_arf_stream_with_eval(
    model_base, X_phase0, y_phase0, X_phase1, y_phase1, X_phase2, y_phase2,
    X_test_encoded, y_test
)
print("After Phase 0 - macro F1:", rep0.get("macro avg", {}).get("f1-score", 0))
print("After Phase 1 - macro F1:", rep1.get("macro avg", {}).get("f1-score", 0))
print("After Phase 2 (final) - macro F1:", rep2.get("macro avg", {}).get("f1-score", 0))
print("\n=== ARF_Base - classification_report (final) ===")
print(classification_report(y_test, predict_arf_batch(model_base, X_test_encoded),
                            labels=TARGET_CLASSES, zero_division=0))

After Phase 0 - macro F1: 0.4709696801393176
After Phase 1 - macro F1: 0.04354494426436872
After Phase 2 (final) - macro F1: 0.003517411185367569

=== ARF_Base - classification_report (final) ===
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00      9711
         DoS       0.00      0.00      0.00      7458
       Probe       0.00      0.00      0.00      2421
         R2L       0.00      0.00      0.00      2754
         U2R       0.01      1.00      0.02       200

    accuracy                           0.01     22544
   macro avg       0.00      0.20      0.00     22544
weighted avg       0.00      0.01      0.00     22544



## 10. Train & evaluate all 7 variants (phase-wise)

Với mỗi biến thể: train theo phase và lưu `after_p0`, `after_p1`, `after_p2` để sau đó tính AA, FM, BWT.

In [10]:
# Cell 10: Train all variants with phase-wise eval; store after_p0, after_p1, after_p2
results = {}
for name, make_model in VARIANTS:
    print("Training", name, "...")
    model = make_model()
    r0, r1, r2 = train_arf_stream_with_eval(
        model, X_phase0, y_phase0, X_phase1, y_phase1, X_phase2, y_phase2,
        X_test_encoded, y_test
    )
    results[name] = {"after_p0": r0, "after_p1": r1, "after_p2": r2}
print("Done. Variants:", list(results.keys()))
print("Each result has keys:", list(results[list(results.keys())[0]].keys()))

Training ARF_Base ...
Training ARF_ADWIN ...
Training ARF_KSWIN ...
Training ARF_PH ...
Training ARF_DDM ...


AttributeError: module 'river.drift' has no attribute 'DDM'

## 11. Summary: F1 per class per variant

In bảng F1 từng lớp cho từng biến thể (macro avg để so sánh nhanh).

In [ ]:
# Cell 11: F1 summary table (final = after_p2)
summary = []
for name, reports in results.items():
    report = reports["after_p2"]
    row = {"variant": name}
    for cls in TARGET_CLASSES:
        row[cls] = report.get(cls, {}).get("f1-score", 0) or 0
    row["macro_avg"] = report.get("macro avg", {}).get("f1-score", 0) or 0
    summary.append(row)
summary_df = pd.DataFrame(summary)
print(summary_df.to_string(index=False))

## 12. AA, FM, BWT – định nghĩa và helper (context §5)

- **AA (Average Accuracy):** Trung bình hiệu năng trên từng task tại thời điểm cuối (sau P2). Task 0 = Normal/DoS/Probe (macro F1), Task 1 = R2L, Task 2 = U2R.
- **FM (Forgetting Measure):** F1 tốt nhất của task (ngay sau khi học xong task) trừ F1 của task tại cuối. FM > 0 = quên.
- **BWT (Backward Transfer):** F1 của task tại cuối trừ F1 ngay sau khi học task đó. BWT < 0 = ảnh hưởng tiêu cực (quên). Tham khảo Task 2 cho logic FM.

In [ ]:
# Cell 12: Helpers for AA, FM, BWT
CLASSES_TASK0 = ["Normal", "DoS", "Probe"]

def f1_for_class(report, cls):
    return report.get(cls, {}).get("f1-score", 0) or 0

def macro_f1_task0(report):
    return np.mean([f1_for_class(report, c) for c in CLASSES_TASK0])

def compute_aa_fm_bwt(reports):
    """reports = dict with keys after_p0, after_p1, after_p2 (each a classification_report dict)."""
    r0, r1, r2 = reports["after_p0"], reports["after_p1"], reports["after_p2"]
    # AA: average over 3 tasks at final (after_p2)
    aa = (macro_f1_task0(r2) + f1_for_class(r2, "R2L") + f1_for_class(r2, "U2R")) / 3.0
    # FM: forgetting on task0 and task1 (task2 has no "after" to forget)
    fm0 = macro_f1_task0(r0) - macro_f1_task0(r2)
    fm1 = f1_for_class(r1, "R2L") - f1_for_class(r2, "R2L")
    fm = (fm0 + fm1) / 2.0
    # BWT: backward transfer (positive = good)
    bwt0 = macro_f1_task0(r2) - macro_f1_task0(r0)
    bwt1 = f1_for_class(r2, "R2L") - f1_for_class(r1, "R2L")
    bwt = (bwt0 + bwt1) / 2.0
    return aa, fm, bwt

# Quick check on first variant
name0 = list(results.keys())[0]
aa0, fm0, bwt0 = compute_aa_fm_bwt(results[name0])
print(f"Example ({name0}): AA={aa0:.4f}, FM={fm0:.4f}, BWT={bwt0:.4f}")

## 13. Bảng AA, FM, BWT theo từng biến thể

In bảng so sánh AA, FM, BWT cho tất cả 7 biến thể (trước/sau khắc phục theo context).

In [ ]:
# Cell 13: AA, FM, BWT table per variant
metrics_rows = []
for name in results:
    aa, fm, bwt = compute_aa_fm_bwt(results[name])
    metrics_rows.append({"variant": name, "AA": aa, "FM": fm, "BWT": bwt})
metrics_df = pd.DataFrame(metrics_rows)
print("=== AA, FM, BWT per variant (context §5) ===")
print(metrics_df.to_string(index=False))
print("\nFM > 0: forgetting; BWT < 0: negative backward transfer.")

In [ ]:
# ARF_Base – Train streaming Phase0 -> Phase1 -> Phase2, evaluate sau mỗi phase

from sklearn.metrics import classification_report
import time


def train_arf_stream_with_eval_base(model,
                                    X_phase0, y_phase0,
                                    X_phase1, y_phase1,
                                    X_phase2, y_phase2,
                                    X_test_encoded, y_test,
                                    log_interval=20000):
    """Train 1 mô hình ARF_Base theo thứ tự P0 -> P1 -> P2.

    Sau mỗi phase, evaluate trên full test 5 lớp bằng classification_report.
    Trả về (report_text_p0, report_text_p1, report_text_p2) để tiện tham chiếu.
    """
    print("=" * 80)
    print("Training ARF_Base with 3 phases (P0 -> P1 -> P2)")
    print("=" * 80)

    # ========== PHASE 0 ==========
    print(f"\n[Phase 0] Training on {len(X_phase0):,} samples (Normal, DoS, Probe)...")
    start = time.time()
    for i in range(len(X_phase0)):
        model.learn_one(row_to_dict(X_phase0[i]), y_phase0[i])
        if (i + 1) % log_interval == 0 or (i + 1) == len(X_phase0):
            elapsed = time.time() - start
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            print(f"  [{i+1:6,}/{len(X_phase0):6,}]  Elapsed: {elapsed:6.1f}s  Rate: {rate:6.0f} samples/s")
    t0 = time.time() - start
    print(f"[Phase 0] Done in {t0:.1f}s")

    print("[Phase 0] Evaluating on full test set...")
    pred0 = predict_arf_batch(model, X_test_encoded)
    rep0_text = classification_report(y_test, pred0, labels=TARGET_CLASSES, zero_division=0)
    print("\n=== ARF_Base - classification_report AFTER PHASE 0 ===")
    print(rep0_text)

    # ========== PHASE 1 ==========
    print(f"\n[Phase 1] Training on {len(X_phase1):,} samples (R2L only)...")
    start = time.time()
    for i in range(len(X_phase1)):
        model.learn_one(row_to_dict(X_phase1[i]), y_phase1[i])
        if (i + 1) % log_interval == 0 or (i + 1) == len(X_phase1):
            elapsed = time.time() - start
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            print(f"  [{i+1:6,}/{len(X_phase1):6,}]  Elapsed: {elapsed:6.1f}s  Rate: {rate:6.0f} samples/s")
    t1 = time.time() - start
    print(f"[Phase 1] Done in {t1:.1f}s")

    print("[Phase 1] Evaluating on full test set...")
    pred1 = predict_arf_batch(model, X_test_encoded)
    rep1_text = classification_report(y_test, pred1, labels=TARGET_CLASSES, zero_division=0)
    print("\n=== ARF_Base - classification_report AFTER PHASE 1 ===")
    print(rep1_text)

    # ========== PHASE 2 ==========
    print(f"\n[Phase 2] Training on {len(X_phase2):,} samples (U2R only)...")
    start = time.time()
    for i in range(len(X_phase2)):
        model.learn_one(row_to_dict(X_phase2[i]), y_phase2[i])
        if (i + 1) % max(1, log_interval // 10) == 0 or (i + 1) == len(X_phase2):
            elapsed = time.time() - start
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            print(f"  [{i+1:4,}/{len(X_phase2):4,}]  Elapsed: {elapsed:6.2f}s  Rate: {rate:6.0f} samples/s")
    t2 = time.time() - start
    print(f"[Phase 2] Done in {t2:.2f}s")

    print("[Phase 2] Evaluating on full test set...")
    pred2 = predict_arf_batch(model, X_test_encoded)
    rep2_text = classification_report(y_test, pred2, labels=TARGET_CLASSES, zero_division=0)
    print("\n=== ARF_Base - classification_report AFTER PHASE 2 ===")
    print(rep2_text)

    print("\n" + "=" * 80)
    print("ARF_Base training timeline:")
    print(f"  Phase 0: {t0:.1f}s   Phase 1: {t1:.1f}s   Phase 2: {t2:.2f}s")
    print("=" * 80)

    return rep0_text, rep1_text, rep2_text


# Khởi tạo ARF_Base và chạy huấn luyện 3 phase
model_base_stream = make_arf_base()
rep0_text, rep1_text, rep2_text = train_arf_stream_with_eval_base(
    model_base_stream,
    X_phase0, y_phase0,
    X_phase1, y_phase1,
    X_phase2, y_phase2,
    X_test_encoded, y_test,
    log_interval=20000,
)